In [ ]:
lulc_rst = '/mnt/d/autocls01/fidx/lulc_ref.tif'

idx_rst = {
    "20220504" : {
        "ndvi" : '/mnt/d/autocls01/fidx/ndvi_20220504.tif',
        "ndbi" : '/mnt/d/autocls01/fidx/ndbi_20220504.tif',
        "ndwi" : '/mnt/d/autocls01/fidx/ndwi_20220504.tif'
    },
    "20220812" : {
        "ndvi" : '/mnt/d/autocls01/fidx/ndvi_20220812.tif',
        "ndbi" : '/mnt/d/autocls01/fidx/ndbi_20220812.tif',
        "ndwi" : '/mnt/d/autocls01/fidx/ndwi_20220812.tif'
    },
    "20221001" : {
        "ndvi" : '/mnt/d/autocls01/fidx/ndvi_20221001.tif',
        "ndbi" : '/mnt/d/autocls01/fidx/ndbi_20221001.tif',
        "ndwi" : '/mnt/d/autocls01/fidx/ndwi_20221001.tif'
    }
}

idx_rules = {
    1 : {
        "ndbi" : {'operator' : '>', 'value': 0, },
        "ndvi" : {'operator' : '<', 'value': 0.3},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    2 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    3 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    4 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    5 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    6 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    7 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    8 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '<', 'value': 0.3},
        "ndwi" : {'operator' : '>', 'value': 0}
    }
}

fraster= '/mnt/d/autocls01/fidx/lulc_filter.tif'

In [ ]:
import os
from osgeo import gdal
import numpy as np
from glass.rd.rst import rst_to_array
from glass.prop.rst import get_nodata
from glass.pys.oss import lst_ff, fprop
from glass.wenv.grs import run_grass
from glass.pys.tm import now_as_str

In [ ]:
# Check if outfolder exists
out_folder = os.path.dirname(fraster)
if not os.path.exists(out_folder):
    mkdir(out_folder, overwrite=None)
    
"""
Start GRASS GIS Session
"""

loc = f'loc_{now_as_str()}'
grsb = run_grass(
    out_folder, grassBIN='grass78', location=loc,
    srs=lulc_rst
    )

import grass.script.setup as gsetup
    
gsetup.init(grsb, out_folder, loc, 'PERMANENT')

In [ ]:
from glass.it.rst import rst_to_grs, grs_to_rst
from grass.pygrass.modules import Module

In [ ]:
# Identificar classes existentes no raster com as classes de LULC

lulc_img = rst_to_array(lulc_rst)

ndval = get_nodata(lulc_rst)

lulc_class = [v for v in np.unique(lulc_img) if v != ndval]

In [ ]:
# Send all rasters to GRASS GIS

lulc_grs = rst_to_grs(lulc_rst, fprop(lulc_rst, 'fn'))

for d in idx_rst:
    for k in idx_rst[d]:
        idx_rst[d][k] = rst_to_grs(idx_rst[d][k], fprop(idx_rst[d][k], 'fn'))

Para cada classe, índice e data, criar um ficheiro que indica se o pixel deve ser considerado para treino (1) ou não (0)

In [ ]:
mask_by_class_day_idx = {}

for cls in lulc_class:
    mask_by_class_day_idx[cls] = {}
    for d in idx_rst:
        mask_by_class_day_idx[cls][d] = {}
        for k in idx_rst[d]:
            if not idx_rules[cls][k]: continue
            
            form = (
                f"if({lulc_grs} == {str(cls)} && "
                f"{idx_rst[d][k]} {idx_rules[cls][k]['operator']} "
                f"{idx_rules[cls][k]['value']}, 1, 0)"
            )
            
            outmask = f'mask_{cls}_{k}_{d}'
            rc = Module ('r.mapcalc', expression=f'{outmask} = {form}')
            
            mask_by_class_day_idx[cls][d][k] = outmask
            
            grs_to_rst(outmask, os.path.join(
                os.path.dirname(fraster),
                f'{outmask}.tif'
            ), is_int=True)

Para cada classe e índice, criar um ficheiro que indica se o pixel deve ser incluído no treino (1) ou não (0), tendo em conta as várias máscaras obtidas para essa classe e índice em cada uma das datas consideradas

Obter máscara binária final para cada classe

Obter ficheiro que indica as classes a considerar no treino - neste raster, o valor 0 indica que o pixel não deve ser considerado no treino.